# Perl Weekly Challenge 305
These are solutions to [Perl Weekly Challenge 305](https://theweeklychallenge.org/blog/perl-weekly-challenge-305/) in [Dyalog APL](https://www.dyalog.com/).

## Task 1: Binary Prefix
Take a Boolean vector as input and return a Boolean vector where a 1 indicates that the partial binary number up to that element is a prime number.

For example, `BinaryPrefix 1 0 1` returns `0 1 1` because:
- $1$ is not prime
- $2=b_2(1 0)$ is prime
- $5=b_2(1 0 1)$ is prime

### Creating prefixes
There are several valid approaches. The simplest is a **catenate-scan** `,\⍵`.

In [5]:
]box on
,\1 0 1 1 0 1 1

Was ON

┌─┬───┬─────┬───────┬─────────┬───────────┬─────────────┐
│1│1 0│1 0 1│1 0 1 1│1 0 1 1 0│1 0 1 1 0 1│1 0 1 1 0 1 1│
└─┴───┴─────┴───────┴─────────┴───────────┴─────────────┘

We can then convert each number with the base conversion primitive.

In [29]:
b←1 0 1 1 0 1 1
↑(b)(2⊥¨,\b)

1 0 1 1 0 1 1
1 2 5 11 22 45 91

It can be more efficient to build a flat array that works directly with primitive functions in APL, rather than looping using the each `F¨` operator. In the following expression, we build a flat Boolean matrix of prefixes using [the rank operator](https://course.dyalog.com/cells-and-axes/#the-rank-operator). The base conversion primitive function can then work on the entire array at once rather than having to loop over individual prefixes. This is significantly faster than `2⊥¨,\b`.

In [32]:
2⊥i⌽(i←⍳≢b)(↑⍤0 1)b

1 2 5 11 22 45 91

### Checking Primes
A direct approach is to use the "unique divisors" idiom.

In [33]:
(∪⊢∨⍳)¨1 2 5 11 22 45 91

┌─┬───┬───┬────┬─────────┬─────────────┬─────────┐
│1│1 2│1 5│1 11│1 2 11 22│1 3 5 9 15 45│1 7 13 91│
└─┴───┴───┴────┴─────────┴─────────────┴─────────┘

We can then count the unique divisors and check which have exactly two, which is the definition of being a prime number.

In [35]:
2=(≢∘∪⊢∨⍳)¨1 2 5 11 22 45 91

0 1 1 1 0 0 0

The `(∪⊢∨⍳)` train is cute and somewhat says exactly what we want: "unique divisors of ⍵". However, it is quite inefficient to find all divisors between every number and the list of integers up to that number.

Instead, let's find where all integers up to the largest in our list are evenly divided with the modulo function.

In [46]:
{2=+⌿0=(⍳⌈/⍵)∘.|⍵}1 2 5 11 22 45 91

0 1 1 1 0 0 0

A yet more efficient prime number checker is found in the `pco` function in the [dfns workspace](https://dfns.dyalog.com), although it is a simple lookup for small primes.

In [36]:
'pco'⎕CY'dfns'
1 pco 1 2 5 11 22 45 91

0 1 1 1 0 0 0

Let's put this all together and try the test cases from the website.

In [47]:
BinaryPrefix ← {2=+⌿0=(⍳⌈/n)∘.|n←2⊥i⌽(i←⍳≢⍵)(↑⍤0 1)⍵}
BinaryPrefix 1 0 1
BinaryPrefix 1 1 0
BinaryPrefix 1 1 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1

0 1 1

0 1 0

0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1

## Task 2: Alien Dictionary
You are given a list of words and alien dictionary character order.

Write a script to sort lexicographically the given list of words based on the alien dictionary characters.

First, let's get our inputs. The Raku language appears to have the `qw` function that creates a list of characters. In APL, that is the default format for character data, so we'll just paste the input from the website and remove the spaces.

In [49]:
alien ← 'h l a b y d e f g i r k m n o p q j s t u v w x c z'~' '

For convenience, we can also take the raw text for the list of strings, swap round with square parentheses, and process it with `⎕JSON`.

In [62]:
words ← 0⎕JSON'["perl", "python", "raku"]'

As for our solution, this is simply one of those unusual times when you actually need dyadic grade.

Dyadic grade takes a simple character vector dictionary left argument `⍺` and a simple character array right argument `⍵` and lexicographically sorts major cells in `⍵` according to the order of elements in `⍺`, leaving those not found in `⍺` in their original order.

The only thing to note is that dyadic grade only works on simple character data, so we must mix our input list of words to make a simple character matrix. However, we can use the grade on the original input list of strings.

In [63]:
alien {⍵[⍺⍋↑⍵]} words

┌────┬──────┬────┐
│raku│python│perl│
└────┴──────┴────┘

And finally, the second example inputs.

In [66]:
alien ← 'c o r l d a b t e f g h i j k m n p q s w u v x y z'~' '
words ← 0⎕JSON'["the", "weekly", "challenge"]'
alien {⍵[⍺⍋↑⍵]} words

┌─────────┬───┬──────┐
│challenge│the│weekly│
└─────────┴───┴──────┘